# Data Preprocessing by Muhammad Hamza Waseem

In [758]:
import os, pandas as pd
import seaborn as sns
import math

In [692]:
# Load individual queries with small dataset
df1 = pd.read_csv("C:/Users/abcal/OneDrive/Desktop/Technocolobs/Optimizing-Delivery-Partner-Allocation-in-Last-Mile-Logistics/Delivery Five Cities Datasets/delivery_cq.csv")
df2 = pd.read_csv("C:/Users/abcal/OneDrive/Desktop/Technocolobs/Optimizing-Delivery-Partner-Allocation-in-Last-Mile-Logistics/Delivery Five Cities Datasets/delivery_hz.csv")
df3 = pd.read_csv("C:/Users/abcal/OneDrive/Desktop/Technocolobs/Optimizing-Delivery-Partner-Allocation-in-Last-Mile-Logistics/Delivery Five Cities Datasets/delivery_delivery_jl.csv")
df4 = pd.read_csv("C:/Users/abcal/OneDrive/Desktop/Technocolobs/Optimizing-Delivery-Partner-Allocation-in-Last-Mile-Logistics/Delivery Five Cities Datasets/delivery_sh.csv")
df5 = pd.read_csv("C:/Users/abcal/OneDrive/Desktop/Technocolobs/Optimizing-Delivery-Partner-Allocation-in-Last-Mile-Logistics/Delivery Five Cities Datasets/delivery_yt.csv")

In [694]:
#combined it
combined_df = pd.concat([df1, df2, df3, df4, df5], ignore_index=True)

In [696]:
# Convert 'ds' to a string and ensure it's zero-padded (e.g., '0813' for August 13)
combined_df['ds'] = combined_df['ds'].astype(str).str.zfill(4)

In [634]:
# Extract month and day
# combined_df_jl_yt['month'] = combined_df_jl_yt['ds'].str[:2]  # First two digits for month
# combined_df_jl_yt['day'] = combined_df_jl_yt['ds'].str[2:]    # Last two digits for day

# # Create a full date assuming the current year (e.g., 2024)
# combined_df_jl_yt['date'] = pd.to_datetime(combined_df_jl_yt['day'] + '-' + combined_df_jl_yt['month'] + '-2024', format='%d-%m-%Y')

In [636]:
# # Ensure the 'date' column is in datetime format
# combined_df_jl_yt['date'] = pd.to_datetime(combined_df_jl_yt['date'])

# # Format the 'date' column and store it in 'ds'
# combined_df_jl_yt['ds'] = combined_df_jl_yt['date'].dt.strftime('%d-%m-%Y')

# # Drop the 'date' column
# combined_df_jl_yt = combined_df_jl_yt.drop(columns=['date'])

In [638]:
# # Drop the 'month' and 'day' column
# combined_df_jl_yt = combined_df_jl_yt.drop(columns=['month'])
# combined_df_jl_yt = combined_df_jl_yt.drop(columns=['day'])

In [698]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4514661 entries, 0 to 4514660
Data columns (total 17 columns):
 #   Column             Dtype  
---  ------             -----  
 0   order_id           int64  
 1   region_id          int64  
 2   city               object 
 3   courier_id         int64  
 4   lng                float64
 5   lat                float64
 6   aoi_id             int64  
 7   aoi_type           int64  
 8   accept_time        object 
 9   accept_gps_time    object 
 10  accept_gps_lng     float64
 11  accept_gps_lat     float64
 12  delivery_time      object 
 13  delivery_gps_time  object 
 14  delivery_gps_lng   float64
 15  delivery_gps_lat   float64
 16  ds                 object 
dtypes: float64(6), int64(5), object(6)
memory usage: 585.6+ MB


In [700]:
print(combined_df['city'].value_counts())

city
Hangzhou     1861600
Shanghai     1483864
Chongqing     931351
Yantai        206431
Jilin          31415
Name: count, dtype: int64


In [704]:
combined_df = combined_df.drop_duplicates(subset='order_id', keep='first')

In [778]:
# Find null (NaN) values
null_values = combined_df.isnull()

# Find empty string values
empty_values = (combined_df == '')

# Combine both null and empty values
null_or_empty = null_values | empty_values

# Print results
print("Null values:\n", null_values)
print("\nEmpty values:\n", empty_values)
print("\nNull or empty values:\n", null_or_empty)

Null values:
          order_id  region_id   city  courier_id    lng    lat  aoi_id  \
0           False      False  False       False  False  False   False   
1           False      False  False       False  False  False   False   
2           False      False  False       False  False  False   False   
3           False      False  False       False  False  False   False   
4           False      False  False       False  False  False   False   
...           ...        ...    ...         ...    ...    ...     ...   
4514656     False      False  False       False  False  False   False   
4514657     False      False  False       False  False  False   False   
4514658     False      False  False       False  False  False   False   
4514659     False      False  False       False  False  False   False   
4514660     False      False  False       False  False  False   False   

         aoi_type  accept_gps_lng  accept_gps_lat  delivery_gps_lng  \
0           False           False     

In [708]:
combined_df.isna().any()

order_id             False
region_id            False
city                 False
courier_id           False
lng                  False
lat                  False
aoi_id               False
aoi_type             False
accept_time          False
accept_gps_time      False
accept_gps_lng        True
accept_gps_lat        True
delivery_time        False
delivery_gps_time    False
delivery_gps_lng     False
delivery_gps_lat     False
ds                   False
dtype: bool

In [710]:
#remove null values
#combined_df.dropna(inplace=True)
combined_df['accept_gps_lng'].isna().sum()

3377

In [712]:
#remove null values
#combined_df.dropna(inplace=True)
combined_df['accept_gps_lat'].isna().sum()

3377

In [716]:
# Fill the missing values of location with the mean of respective columns
combined_df['accept_gps_lng']=combined_df['accept_gps_lng'].fillna(combined_df['accept_gps_lng'].mean())
combined_df['accept_gps_lat']=combined_df['accept_gps_lat'].fillna(combined_df['accept_gps_lat'].mean())

In [718]:
combined_df['accept_gps_lat'].isna().sum()
combined_df['accept_gps_lng'].isna().sum()

0

In [722]:
combined_df['aoi_type'] = pd.Categorical(combined_df['aoi_type'], ordered=False)

In [732]:
# Split the 'accept_time' column by space into two columns: 'date' and 'time'
combined_df[['accept_date', 'accept_time1']] = combined_df['accept_time'].str.split(' ', expand=True)

# Drop the original 'accept_time' column
combined_df = combined_df.drop(columns=['accept_time'])

# Split the 'accept_gps_time' column by space into two columns: 'date' and 'time'
combined_df[['accept_gps_date', 'accept_gps_time1']] = combined_df['accept_gps_time'].str.split(' ', expand=True)

# Drop the original 'accept_gps_time'
combined_df = combined_df.drop(columns=['accept_gps_time'])

# Split the 'delivery_time' column by space into two columns: 'date' and 'time'
combined_df[['delivery_date', 'delivery_time1']] = combined_df['delivery_time'].str.split(' ', expand=True)

# Split the 'delivery_gps_time' column by space into two columns: 'date' and 'time'
combined_df[['delivery_gps_date', 'delivery_gps_time1']] = combined_df['delivery_gps_time'].str.split(' ', expand=True)

# Drop the original 'delivery_time' column 
combined_df = combined_df.drop(columns=['delivery_time'])

# Drop the original 'delivery_gps_time' column
combined_df = combined_df.drop(columns=['delivery_gps_time'])

# Create a full date assuming the current year (e.g., 2024)
combined_df['accept_date'] = pd.to_datetime(combined_df['accept_date'] + '-2024', format='%m-%d-%Y')

# Convert 'accept_date' column to datetime and reformat
combined_df['accept_date'] = pd.to_datetime(combined_df['accept_date'], format='%m-%d-%Y').dt.strftime('%d-%m-%Y')

# Create a full date assuming the current year (e.g., 2024)
combined_df['accept_gps_date'] = pd.to_datetime(combined_df['accept_gps_date'] + '-2024', format='%m-%d-%Y')

# Convert 'accept_gps_date' column to datetime and reformat
combined_df['accept_gps_date'] = pd.to_datetime(combined_df['accept_gps_date'], format='%m-%d-%Y').dt.strftime('%d-%m-%Y')

# Create a full date assuming the current year (e.g., 2024)
combined_df['delivery_date'] = pd.to_datetime(combined_df['delivery_date'] + '-2024', format='%m-%d-%Y')

# Convert 'delivery_date' column to datetime and reformat
combined_df['delivery_date'] = pd.to_datetime(combined_df['delivery_date'], format='%m-%d-%Y').dt.strftime('%d-%m-%Y')

# Create a full date assuming the current year (e.g., 2024)
combined_df['delivery_gps_date'] = pd.to_datetime(combined_df['delivery_gps_date'] + '-2024', format='%m-%d-%Y')

# Convert 'delivery_gps_date' column to datetime and reformat
combined_df['delivery_gps_date'] = pd.to_datetime(combined_df['delivery_gps_date'], format='%m-%d-%Y').dt.strftime('%d-%m-%Y')

In [736]:
combined_df['accept_time'] = combined_df['accept_date'] + ' ' + combined_df['accept_time1']
combined_df['accept_gps_time'] = combined_df['accept_gps_date'] + ' ' + combined_df['accept_gps_time1']
combined_df['delivery_time'] = combined_df['delivery_date'] + ' ' + combined_df['delivery_time1']
combined_df['delivery_gps_time'] = combined_df['delivery_gps_date'] + ' ' + combined_df['delivery_gps_time1']

In [740]:
combined_df = combined_df.drop(columns=['accept_time1'])
combined_df = combined_df.drop(columns=['accept_gps_time1'])
combined_df = combined_df.drop(columns=['delivery_time1'])
combined_df = combined_df.drop(columns=['delivery_gps_time1'])
combined_df = combined_df.drop(columns=['accept_date'])
combined_df = combined_df.drop(columns=['accept_gps_date'])
combined_df = combined_df.drop(columns=['delivery_date'])
combined_df = combined_df.drop(columns=['delivery_gps_date'])

In [746]:
combined_df['delivery_time'] = pd.to_datetime(combined_df['delivery_time'], dayfirst=True)

In [750]:
combined_df['accept_time']=pd.to_datetime(combined_df['accept_time'], dayfirst=True)
combined_df['accept_gps_time'] = pd.to_datetime(combined_df['accept_gps_time'], dayfirst=True)
combined_df['delivery_gps_time']=pd.to_datetime(combined_df['delivery_gps_time'], dayfirst=True)
combined_df['ds'] = combined_df['ds'].astype(int)

In [752]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4514661 entries, 0 to 4514660
Data columns (total 17 columns):
 #   Column             Dtype         
---  ------             -----         
 0   order_id           int64         
 1   region_id          int64         
 2   city               object        
 3   courier_id         int64         
 4   lng                float64       
 5   lat                float64       
 6   aoi_id             int64         
 7   aoi_type           category      
 8   accept_gps_lng     float64       
 9   accept_gps_lat     float64       
 10  delivery_gps_lng   float64       
 11  delivery_gps_lat   float64       
 12  ds                 int32         
 13  accept_time        datetime64[ns]
 14  accept_gps_time    datetime64[ns]
 15  delivery_time      datetime64[ns]
 16  delivery_gps_time  datetime64[ns]
dtypes: category(1), datetime64[ns](4), float64(6), int32(1), int64(4), object(1)
memory usage: 538.2+ MB


In [754]:
combined_df['delivery_duration'] = combined_df['delivery_time'] - combined_df['accept_time']

In [760]:
combined_df['distance'] = combined_df.apply(
    lambda row: math.sqrt(
        (row['delivery_gps_lng'] - row['accept_gps_lng'])**2 +
        (row['delivery_gps_lat'] - row['accept_gps_lat'])**2
    ),
    axis=1
)

In [774]:
# Find invalid Longitude and Latitude

df_invalid = combined_df[~((combined_df['delivery_gps_lat'].between(-90, 90)) & (combined_df['delivery_gps_lng'].between(-180, 180)))]
print(df_invalid)

Empty DataFrame
Columns: [order_id, region_id, city, courier_id, lng, lat, aoi_id, aoi_type, accept_gps_lng, accept_gps_lat, delivery_gps_lng, delivery_gps_lat, ds, accept_time, accept_gps_time, delivery_time, delivery_gps_time, delivery_duration, distance]
Index: []


In [776]:
# Calculate Q1, Q3, and IQR
Q1 = combined_df['delivery_duration'].quantile(0.25)
Q3 = combined_df['delivery_duration'].quantile(0.75)
IQR = Q3 - Q1

# Define bounds for outliers
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
df_clean = combined_df[(combined_df['delivery_duration'] >= lower_bound) & (combined_df['delivery_duration'] <= upper_bound)]

df_clean

,order_id,region_id,city,courier_id,lng,lat,aoi_id,aoi_type,accept_gps_lng,accept_gps_lat,delivery_gps_lng,delivery_gps_lat,ds,accept_time,accept_gps_time,delivery_time,delivery_gps_time,delivery_duration,distance
0,2031782,10,Chongqing,73,108.71571,30.90228,50,14,108.71826,30.95587,108.66361,30.96702,1022,2024-10-22 10:26:00,2024-10-22 10:26:00,2024-10-22 17:04:00,2024-10-22 17:04:00,0 days 06:38:00,0.055776
6,4481765,10,Chongqing,3605,108.71605,30.90410,50,14,108.71824,30.95583,108.71429,30.90416,930,2024-09-30 10:00:00,2024-09-30 10:00:00,2024-09-30 16:38:00,2024-09-30 16:38:00,0 days 06:38:00,0.051821
20,3098203,10,Chongqing,1635,108.71797,30.94364,296,14,108.71801,30.95637,108.71809,30.94260,710,2024-07-10 08:33:00,2024-07-10 08:33:00,2024-07-10 13:24:00,2024-07-10 13:24:00,0 days 04:51:00,0.013770
21,356619,10,Chongqing,1635,108.71979,30.94130,296,14,108.71803,30.95629,108.71970,30.94235,909,2024-09-09 09:04:00,2024-09-09 09:04:00,2024-09-09 10:49:00,2024-09-09 10:49:00,0 days 01:45:00,0.014040
22,1484207,10,Chongqing,1635,108.72106,30.94164,296,14,108.71820,30.95598,108.72307,30.94201,1019,2024-10-19 08:29:00,2024-10-19 08:29:00,2024-10-19 10:11:00,2024-10-19 10:11:00,0 days 01:42:00,0.014795
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4514655,3127175,167,Yantai,1571,121.19641,36.78106,60137,1,121.20557,36.76352,121.17895,36.78380,512,2024-05-12 09:10:00,2024-05-12 09:10:00,2024-05-12 13:03:00,2024-05-12 13:03:00,0 days 03:53:00,0.033465
4514656,2512759,167,Yantai,1571,121.19537,36.78027,60137,1,121.20571,36.76358,121.19512,36.78016,703,2024-07-03 15:14:00,2024-07-03 15:14:00,2024-07-03 17:33:00,2024-07-03 17:33:00,0 days 02:19:00,0.019673
4514657,3779171,167,Yantai,1571,121.19646,36.78103,60137,1,121.20567,36.76358,121.19635,36.78090,524,2024-05-24 10:06:00,2024-05-24 10:06:00,2024-05-24 15:12:00,2024-05-24 15:12:00,0 days 05:06:00,0.019668
4514658,4198558,167,Yantai,3402,121.19515,36.78057,60137,1,121.20555,36.76358,121.19567,36.78372,501,2024-05-01 15:13:00,2024-05-01 15:13:00,2024-05-01 16:06:00,2024-05-01 16:06:00,0 days 00:53:00,0.022433


In [762]:
combined_df

,order_id,region_id,city,courier_id,lng,lat,aoi_id,aoi_type,accept_gps_lng,accept_gps_lat,delivery_gps_lng,delivery_gps_lat,ds,accept_time,accept_gps_time,delivery_time,delivery_gps_time,delivery_duration,distance
0,2031782,10,Chongqing,73,108.71571,30.90228,50,14,108.71826,30.95587,108.66361,30.96702,1022,2024-10-22 10:26:00,2024-10-22 10:26:00,2024-10-22 17:04:00,2024-10-22 17:04:00,0 days 06:38:00,0.055776
1,4285071,10,Chongqing,3605,108.71639,30.90269,50,14,108.71791,30.95635,108.71644,30.90266,907,2024-09-07 10:13:00,2024-09-07 10:13:00,2024-09-09 15:44:00,2024-09-09 15:44:00,2 days 05:31:00,0.053710
2,4056800,10,Chongqing,3605,108.71645,30.90259,50,14,108.71798,30.95635,108.71647,30.90251,626,2024-06-26 09:49:00,2024-06-26 09:49:00,2024-06-27 16:03:00,2024-06-27 16:03:00,1 days 06:14:00,0.053861
3,3589481,10,Chongqing,3605,108.71650,30.90347,50,14,108.71823,30.95596,108.71650,30.90341,911,2024-09-11 11:01:00,2024-09-11 11:01:00,2024-09-13 17:14:00,2024-09-13 17:14:00,2 days 06:13:00,0.052578
4,2752329,10,Chongqing,3605,108.71608,30.90409,50,14,108.71820,30.95598,108.71413,30.90397,1001,2024-10-01 09:52:00,2024-10-01 09:52:00,2024-10-01 18:30:00,2024-10-01 18:30:00,0 days 08:38:00,0.052169
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4514656,2512759,167,Yantai,1571,121.19537,36.78027,60137,1,121.20571,36.76358,121.19512,36.78016,703,2024-07-03 15:14:00,2024-07-03 15:14:00,2024-07-03 17:33:00,2024-07-03 17:33:00,0 days 02:19:00,0.019673
4514657,3779171,167,Yantai,1571,121.19646,36.78103,60137,1,121.20567,36.76358,121.19635,36.78090,524,2024-05-24 10:06:00,2024-05-24 10:06:00,2024-05-24 15:12:00,2024-05-24 15:12:00,0 days 05:06:00,0.019668
4514658,4198558,167,Yantai,3402,121.19515,36.78057,60137,1,121.20555,36.76358,121.19567,36.78372,501,2024-05-01 15:13:00,2024-05-01 15:13:00,2024-05-01 16:06:00,2024-05-01 16:06:00,0 days 00:53:00,0.022433
4514659,1805034,167,Yantai,3402,121.19631,36.78100,60137,1,121.20561,36.76357,121.19639,36.78082,505,2024-05-05 09:22:00,2024-05-05 09:22:00,2024-05-05 17:24:00,2024-05-05 17:24:00,0 days 08:02:00,0.019559


In [782]:
combined_df.to_csv('C:/Users/abcal/OneDrive/Desktop/Technocolobs/Optimizing-Delivery-Partner-Allocation-in-Last-Mile-Logistics/cleaned_delivery_combined.csv', index=False)